In [2]:
from selenium import webdriver
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
import re 
import requests

In [ ]:
boptions = webdriver.ChromeOptions()
boptions.add_argument("--enable-javascript")
driver = webdriver.Chrome(options=boptions)
driver.get('https://www.nytimes.com/search?dropmab=false&lang=en&query=&sections=Business%7Cnyt%3A%2F%2Fsection%2F0415b2b0-513a-5e78-80da-21ab770cb753%2CWorld%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=newest&types=article')
data = []
last_i = -1

#  A very realistic User-Agent string. Find one from your own browser for best results
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",  # Important: Let them know you can handle compression
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "max-age=0",
}

while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    containers = soup.find_all("li", attrs={"class": "css-1l4w6pd"})

    for i, container in enumerate(containers[last_i:]):

        if i < last_i:
            continue

        container_inner = container.find('div').find('div')
        container_a = container_inner.find('a')

        header = container_a.find('h4')
        header_text = header.text
        href = container_a['href']
        if not href.startswith("http"):
            href = 'https://www.nytimes.com' + href

        brief_el = container_a.find('p', attrs={'class': 'css-e5tzus'})
        brief = "None" if brief_el is None else brief_el.text

        # open the page
        page = requests.get(href, headers=headers, timeout=10)
        soup_inner = BeautifulSoup(page.text, 'html.parser')
        #print(page.text)
        dttime_el = soup_inner.find('time')
        #time.sleep(1)
        #print(href, dttime_el)
        dttime = dttime_el['datetime'] if dttime_el is not None else "None"
        #author = "NOT FOUND"
        tags_element = soup_inner.find('div', attrs={'class': 'css-b9twaf e9ne27s0'}) 
        if tags_element:
            tags_element = tags_element.find_all('a')
            tags = ",".join([el.text for el in tags_element])
        else:
            tags = "None"


        #//*[@id="content"]/div/div[1]/div/div[3]

        print(i, header_text, dttime, href, brief, tags)
        data.append((header_text, dttime, href, brief, tags)) 

        last_i = i

        #data.append((header, timedata, ref))
        #print(f"Found header: {header}, date: {timedata}, ref: {ref}")

    #//*[@id="content"]/div/div[1]/div/div[3]
    bottom = driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[2]/div[2]/div/button')
    location = bottom.location
    size = bottom.size
    action = ActionChains(driver) 
    action.move_to_element(bottom).perform()
    driver.execute_script("arguments[0].click();", bottom)
    time.sleep(1)


0 Gulf Deal-Making Spree Also Benefited Elon Musk and His Family 2025-05-20T15:44:36.272Z https://www.nytimes.com/2025/05/20/world/middleeast/gulf-deal-making-spree-also-benefited-elon-musk-and-his-family.html?searchResultPosition=10 The world’s richest man inked new deals as he tagged along on President Trump’s tour of the Gulf. None
0 Are You a European in a Housing Crunch? We Want to Hear From You. 2025-05-20T10:34:44-04:00 https://www.nytimes.com/2025/05/20/business/europe-housing-crunch.html?searchResultPosition=1 To help us report on the housing crisis in Europe, we want to learn about the housing pressures you are dealing with, how they are affecting your community and how they are being solved. None
1 The E.U. Is Still Ramping Up Sanctions on Russia. Here’s How. 2025-05-20T09:43:14-04:00 https://www.nytimes.com/2025/05/20/world/europe/european-union-russia-sanctions.html?searchResultPosition=2 The European Union has now targeted Moscow’s fleet of covert oil tankers and plans mo

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.114)
Stacktrace:
	GetHandleVerifier [0x00007FF74B3FCF65+75717]
	GetHandleVerifier [0x00007FF74B3FCFC0+75808]
	(No symbol) [0x00007FF74B1C8F9A]
	(No symbol) [0x00007FF74B1B4E35]
	(No symbol) [0x00007FF74B1D9DB4]
	(No symbol) [0x00007FF74B24EE75]
	(No symbol) [0x00007FF74B26ECC2]
	(No symbol) [0x00007FF74B247153]
	(No symbol) [0x00007FF74B210421]
	(No symbol) [0x00007FF74B2111B3]
	GetHandleVerifier [0x00007FF74B6FD74D+3223469]
	GetHandleVerifier [0x00007FF74B6F7CF2+3200338]
	GetHandleVerifier [0x00007FF74B715B23+3322755]
	GetHandleVerifier [0x00007FF74B416A3A+180890]
	GetHandleVerifier [0x00007FF74B41E13F+211359]
	GetHandleVerifier [0x00007FF74B4052B4+109332]
	GetHandleVerifier [0x00007FF74B405462+109762]
	GetHandleVerifier [0x00007FF74B3EBA79+4825]
	BaseThreadInitThunk [0x00007FFBADB87374+20]
	Rtl

In [ ]:
df = pd.DataFrame(data, columns = ['header', 'date', 'link', 'brief', 'tags'])
df = df.drop_duplicates()
df.head()

,header,date,link,brief,tags
590,Inside a Union’s Fight Against Trump’s Federal...,None,https://www.nytimes.com/2025/04/20/business/ec...,Leaders of the union representing government w...,None
591,China Is Finding Ways to Replace American Farmers,None,https://www.nytimes.com/2025/04/20/business/ta...,China has long relied on the U.S. for soybeans...,None
592,Ukraine Says Russia Broke Its Own Easter Cease...,None,https://www.nytimes.com/2025/04/20/world/europ...,President Zelensky of Ukraine said the front l...,None
593,The Trade Adviser Who Hates Trade,None,https://www.nytimes.com/2025/04/20/business/ec...,"Once sidelined, President Trump’s counselor Pe...",None
594,"At World Expo, Japan Recalls a Faded Dynamism",None,https://www.nytimes.com/2025/04/20/business/ja...,The event is stirring memories of an exhibitio...,None
